In [8]:
import plotly.express as px
import pandas as pd
import plotly.offline as pyo
#import qgrid
#qgrid.enable()

In [9]:
def color_scale(value):
    if value < 50:
        return "#FF0000"#red
    elif value >= 50 and value <= 80:
        return "#FFFF00" #yellow
    elif value > 80:
        return "#00FF00" #green

In [11]:
df = pd.read_csv('DataModelCoverage.csv')
#print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'DataModelCoverage.csv'

In [ ]:
dfmean=df.groupby(['Country'])['Coverage'].mean()
print(dfmean)
dfmean.to_csv("avgpercountry.csv", index=True)

Country
Aus             50.892857
De              48.809524
FR              50.595238
ISR             52.083333
IT              50.000000
South Africa    50.297619
Name: Coverage, dtype: float64


In [ ]:
df['color']=df['Coverage'].apply(color_scale)
fig=px.sunburst(df, color='color', values='Coverage', path=['Region','Country','Brand','Model','Coverage'],hover_name='Country')
pyo.plot(fig, auto_open=True, output_type='file', filename='sunburst.html')

'sunburst.html'

In [ ]:
#px.choropleth(dfmean, locations=['Country'], color= ['Coverage'],color_continuous_scale='Viridis', title='Coverage Dashboard')
px.sunburst(df, color='color', values='Coverage', path=['Region','Country','Brand','Model','Coverage'],hover_name='Country')

NameError: name 'df' is not defined

In [ ]:
#print(df)

     Region       Country     Brand Model  Coverage  Group1  Group2  Group3  \
0    Europe            De      Audi    A1     75.00      75     100      75   
1    Europe            De      Audi    A2     62.50      50      75      75   
2    Europe            De      Audi    A3     50.00      50      75      25   
3    Europe            De      Audi    A4     62.50      75      50      75   
4    Europe            De      Audi    A6     62.50      75      50      50   
..      ...           ...       ...   ...       ...     ...     ...     ...   
121  Africa  South Africa  Mercedes  C100     25.00      25      50       0   
122  Africa  South Africa  Mercedes  E500     50.00      75      50      25   
123  Africa  South Africa  Mercedes  C230     75.00      75      75      75   
124  Africa  South Africa  Mercedes   EQS     43.75      50      50      50   
125  Africa  South Africa  Mercedes   EQE     50.00       0      50      75   

     Group4  Data 1 1  ...  Data 2 4  Data 3 1  Dat

In [ ]:
#Trying to show the df in an excel similar way but didnt work.
#qgrid_widget=qgrid.show_grid(df, show_toolbar=True)
#qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…